<a href="https://colab.research.google.com/github/RoronoaZoro-0-7/GPT/blob/main/Large_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets matplotlib numpy jupyter ipykernel tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
!python --version

Python 3.12.12


In [ ]:
!pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.12/dist-packages/functorch/*
    /usr/local/lib/python3.12/dist-packages/torch-2.9.0+cu126.dist-info/*
    /usr/local/lib/python3.12/dist-packages/torch/*
    /usr/local/lib/python3.12/dist-packages/torchgen/*
Proceed (Y/n)? y
y
y
y
y
y
y
yy
y
y
yy
y
y
y
y
yy
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Would remove:
    /usr/local/lib/python3.12/dist-packages/torchvision-0.24.0+cu126.dist-info/*
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libcudart.45e7f3ed.so.12
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libjpeg.37781fad.so.8
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libnvjpeg.e5f20359.so.12
    /usr/local/lib/python3.12/dist-packages/torchvision.libs/libpng1

In [ ]:
import transformers
print(transformers.__version__)

4.57.3


In [ ]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Torch version: 2.5.1+cu121
CUDA available: True
CUDA device name: Tesla T4


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
block_size = 8
batch_size = 4

In [ ]:
with open('wizard of oz.txt','r',encoding='utf-8') as f:
  text = f.read()
print(sorted(set(text)))
chars = sorted(set(text))
chars[61]

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


'd'

In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encoded = lambda s : [string_to_int[c] for c in s]
decoded = lambda s : "".join([int_to_string[i] for i in s])

encoded("hello")
decoded([2,3,4,74,2])

'!"#q!'

In [ ]:
data = torch.tensor(encoded(text),dtype=torch.long)
print(data.shape)

torch.Size([252022])


In [ ]:
n = (int)(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
  data = train_data if split == "train" else test_data
  ix = torch.randint(len(data) - block_size,(batch_size,))
  # all the values in ix represent start from index i
  # these indexes are used to train the sequence
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device),y.to(device)
  return x,y

x,y = get_batch("train")
print(x)
print(y)

tensor([138010,   2851,  30976,  73767])
tensor([[ 1, 63, 75, 72, 70,  1, 77, 65],
        [80, 58, 75, 61,  3, 28,  1, 53],
        [63, 78, 76, 62, 61, 14,  1, 30],
        [61,  1, 61, 72,  1, 58, 69, 69]], device='cuda:0')
tensor([[63, 75, 72, 70,  1, 77, 65, 62],
        [58, 75, 61,  3, 28,  1, 53, 72],
        [78, 76, 62, 61, 14,  1, 30, 78],
        [ 1, 61, 72,  1, 58, 69, 69,  1]], device='cuda:0')


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target is {target}")

when input is tensor([91]) the target is 48
when input is tensor([91, 48]) the target is 65
when input is tensor([91, 48, 65]) the target is 62
when input is tensor([91, 48, 65, 62]) the target is 1
when input is tensor([91, 48, 65, 62,  1]) the target is 44
when input is tensor([91, 48, 65, 62,  1, 44]) the target is 75
when input is tensor([91, 48, 65, 62,  1, 44, 75]) the target is 72
when input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) the target is 67


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# initialize an embedding layer
vocab_size = len(chars)
embedding_dimension = 100

embedding = nn.Embedding(vocab_size,embedding_dimension)

inputs = torch.LongTensor([1,5,3,2])
print(type(embedding(inputs)))

<class 'torch.Tensor'>


In [ ]:
a = torch.tensor([[1,2],[3,4],[5,6]])
b = torch.tensor([[1,2,3],[4,5,6]])
# print(a * b) # element wise operations are used with +,-,*
print(a @ b)# row and column wise operations are done by @
print(a.matmul(b))

tensor([[ 9, 12, 15],
        [19, 26, 33],
        [29, 40, 51]])
tensor([[ 9, 12, 15],
        [19, 26, 33],
        [29, 40, 51]])


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)  # (B,T,C)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(index)
            logits = logits[:, -1, :]   # (B,C)
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, next_token), dim=1)
        return index

model = BigramLanguageModel(vocab_size).to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=500)[0].tolist()
print(decoded(generated))


—[xTX™XY
hzyiqa﻿'”
.KsB$pvKLJrp/’djwmeHRc/xT8ujOQ—l$R Zop]K2﻿2ZqGbS﻿/’o0n3TwsI%i8)Ma5YJq‘™X,)vq•6b(dK—ZpGS;fiH6.ZGJ$Cl—tDutL$ubgaNv_o™*t[“k;1Il,(CMO]Kt#xE‘wv'“G(3M6i™r﻿3qg5vHu•‘f!4g#U2Xqr““72.Hyb?G•”XpuA
ll“w•64g ;%aE2-Vhfy“ySd/?M k.ZDkz&Z#(%4Qd6rp 2xgazFeT[i$MP/&X﻿T“c'%D)NB9CthW?cE‘w7Ek;“),UD‘‘Esu_SsxT•8sN$-;7H rZmMMClkDav8F)Tia—9?• 2D(]B"Bpb$ChL:M5Ib0]BULsgg#Wg™%QeT•;:‘88oO(”_Y?dqzgfsG™o_O4bP‘i8JJ85E3?oReP‘HZRRP9]Yml 6™( ,O)zz?s7f%_#:Z7s7dfbP8p6CQes—ph0FL,JL.o(;-AcRrheUhUKVEHcvGVsBkb0i*GS6hf]_
